In [31]:
# Load packages used in this notebook
import os
import json
import pandas as pd

from cmdstanpy import CmdStanModel, cmdstan_path, install_cmdstan

import arviz as az

import bebi103

# Glicko2

Final project for Alp Küçükelbir's Machine Learning Probabilistic Programming (COMS6998) course.

**Box's Loop**

![Box's Loop](./boxsloop.png)


## 1. Data

## 2. Model

In [ ]:
1. MCMC convergence diagnosis (rhat, pacf, acf, trace plots etc..) - compact şekilde yap talo yap rhat avg koy vb... (mert)
2. VI (OZAN)
3. farklı inference algo için test metriclerini değerlendirmek (ozan)
4. MAPi run et Orjinal inference algoritması check et - ekle convergence a bak (mert)
orjinal infernece algoritmasını nasıl crit edeceğiz? convergence bakabiliriz belki?

ilk set - convergence diagnositcs
ikinci set - kapıştırma

The model we work is based on a popular Bayesian statistical model called Glicko2, an extension of Glicko model, which is the first Bayesian rating model that models uncertainity of player merits in a time dependent manner. In Glicko2, the latent merit/skills of each player are drawn from a Normal distribution, and the variance of normal distribution are drawn from lognormal distribution. Morover, the probability of winning is modeled by a logistic curve. 

In [3]:
glicko_stan = './glicko/glicko.stan'

chess_data = './glicko/chess.data.json'

In [5]:
glicko_stan =  CmdStanModel(stan_file=glicko_stan)

glicko_fit = glicko_stan.sample(
    data=chess_data,
    seed=123)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/mk4139/Glicko2_/glicko/glicko
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 2


In [35]:

import cmdstanpy

glicko_stan = './glicko/glicko.stan'

chess_data = './glicko/chess.data.json'

with bebi103.stan.disable_logging():
    sm = CmdStanModel(stan_file=glicko_stan)

with bebi103.stan.disable_logging():
    samples = sm.sample(
        data=chess_data, chains=4, iter_sampling=1000, adapt_delta=0.95, seed=3252
    )


In [41]:
samples = az.from_cmdstanpy(
    samples, posterior_predictive=["gamma[2,2]"], 
    log_likelihood="log_lik"
)

AttributeError: 'InferenceData' object has no attribute 'column_names'

In [37]:
samples

Inference data with groups:
	> posterior
	> sample_stats

In [38]:
samples.sample_stats.diverging

<xarray.DataArray 'diverging' (chain: 4, draw: 1000)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999

In [39]:

import numpy as np

print(np.sum(samples.sample_stats.diverging))

<xarray.DataArray 'diverging' ()>
array(141)


In [40]:
import bokeh
from bokeh.io import output_notebook, show
output_notebook()

bokeh.io.show(
    bebi103.viz.trace(samples, parameters=["gamma[1,1]", "gamma[1,2]"])
)


Loading BokehJS ...

In [44]:
az.summary(samples)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
"gamma[0,0]",0.122,0.840,-1.464,1.752,0.014,0.014,3405.0,1781.0,3591.0,2234.0,1.00
"gamma[0,1]",-0.090,0.820,-1.620,1.470,0.015,0.013,3117.0,2030.0,3205.0,2464.0,1.00
"gamma[0,2]",-0.009,0.814,-1.639,1.398,0.012,0.015,4403.0,1501.0,4838.0,2509.0,1.00
"gamma[0,3]",0.003,0.819,-1.539,1.573,0.013,0.014,4184.0,1758.0,4414.0,2544.0,1.00
"gamma[0,4]",0.047,0.834,-1.554,1.569,0.014,0.015,3533.0,1634.0,3844.0,2470.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...
"sigma_sq[9,11]",3.054,12.032,0.000,9.783,0.495,0.350,591.0,591.0,334.0,609.0,1.01
"sigma_sq[9,12]",5.708,38.831,0.000,14.875,1.939,1.372,401.0,401.0,364.0,310.0,1.01
tau_sq,0.453,0.233,0.138,0.863,0.019,0.013,156.0,156.0,171.0,309.0,1.02
beta,0.570,0.943,-1.143,2.354,0.025,0.019,1417.0,1259.0,1469.0,1483.0,1.00


In [43]:
print(az.rhat(samples))

<xarray.Dataset>
Dimensions:         (gamma_dim_0: 10, gamma_dim_1: 13, sigma_sq_dim_0: 10, sigma_sq_dim_1: 13)
Coordinates:
  * gamma_dim_0     (gamma_dim_0) int64 0 1 2 3 4 5 6 7 8 9
  * gamma_dim_1     (gamma_dim_1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * sigma_sq_dim_0  (sigma_sq_dim_0) int64 0 1 2 3 4 5 6 7 8 9
  * sigma_sq_dim_1  (sigma_sq_dim_1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    gamma           (gamma_dim_0, gamma_dim_1) float64 1.004 1.002 ... 1.006
    sigma_sq        (sigma_sq_dim_0, sigma_sq_dim_1) float64 1.002 ... 1.011
    tau_sq          float64 1.023
    beta            float64 1.003
    rho             float64 1.023


## 3. Inference

### 1. MCMC

In [ ]:
glicko_fit = glicko_stan.sample(data=chess_data, seed=123)

### 2. VI

## 4. Criticism

In [7]:
import numpy as np

In [10]:
!pip install numpy